# Seat Assignment

Create a software that performs allocation of staff to seats in the company's office trying to group people allocated to the same projects to seat together, but also enforcing some restriction rules.

## Problem definition (from Kory)

1. Same team should seat together
2. Minimize the number of displacements in current seat assignment when projects change
3. Avoid standing desks on Corners of Aisle
4. No more than 2 standing desks per row
5. Some people have fixed seats (e.g. partners)
6. Rows should have 3 or 4 people allocated (tops)
7. People can manifest a preference for more quiet seats (e.g. Brooklyn: C > 3 > 5)
8. People that should be split apart in different floors (e.g. at least someone from ops in each floor)
9. Vacation info
10. Visitors

Spreadsheet: https://docs.google.com/spreadsheets/d/1olVWpxeUghcEgNSW4ZNBDN07eu3dN0iv5L9aki1dVvk/edit#gid=0

# Solution Draft (w.i.p.)

## Data Model

The office seats should be represented as a 2D matrix (could be an adjacency list) of distances. 
Every rows and columns represent seat IDs, and the cell values are a *logical* distance function between seats.

Example of distance matrix represented as adjancency list using nested dicts
TODO: We still need to define the *logical* distance function and compute the seat map from the spreadsheet

In [4]:
# c<Id> = seat
AISLE = '_'
seat_layout = [['c1','c2','c3','c4'], # first table
               ['c5','c6','c7','c8'], # adjacent table
               [AISLE],                 # corridor, aisle
               ['c9','c10','c11','c12'],
               ['c13','c14','c15','c16']]

In [55]:
class Seat(object):
    def __init__(self, _id, row, col):
        self.id = _id
        self.row = row
        self.col = col
    def __str__(self):
        return repr(self)
    def __repr__(self):
        return "Seat('{0}', {1}, {2})".format(self.id, self.row, self.col)
        
def build_seat_coordinates(layout):
    coordinate_map = {}
    aisle_counter =0
    for table_index, seat_list in enumerate(seat_layout):
        if seat_list[0]== AISLE:
            aisle_counter += 1
            continue
        for seat_index,seat_id in enumerate(seat_list):
            coordinate_map[seat_id] = Seat(_id=seat_id,
                                           row=table_index+aisle_counter,
                                           col=seat_index)
            
            
    return coordinate_map

In [56]:
coord_map = build_seat_coordinates(seat_layout)

In [57]:
coord_map # similar to cartesian coordinates

{'c1': Seat('c1', 0, 0),
 'c10': Seat('c10', 4, 1),
 'c11': Seat('c11', 4, 2),
 'c12': Seat('c12', 4, 3),
 'c13': Seat('c13', 5, 0),
 'c14': Seat('c14', 5, 1),
 'c15': Seat('c15', 5, 2),
 'c16': Seat('c16', 5, 3),
 'c2': Seat('c2', 0, 1),
 'c3': Seat('c3', 0, 2),
 'c4': Seat('c4', 0, 3),
 'c5': Seat('c5', 1, 0),
 'c6': Seat('c6', 1, 1),
 'c7': Seat('c7', 1, 2),
 'c8': Seat('c8', 1, 3),
 'c9': Seat('c9', 4, 0)}

In [60]:
from itertools import combinations
from math import sqrt,pow

def distances_from_coordinates(coord_map):
    distance_map = {}
    all_seats = coord_map.values()
    for src,dst in combinations(all_seats, 2):
        distance = sqrt(abs(src.row - dst.row)**2 + abs(src.col - dst.col)**2)
        try:
            distance_map[src.id][dst.id]=distance
        except KeyError:
            distance_map[src.id]={dst.id: distance}
    return distance_map

distance_map = distances_from_coordinates(coord_map)

In [62]:
distance_map['c1']

{'c10': 4.123105625617661,
 'c11': 4.47213595499958,
 'c12': 5.0,
 'c13': 5.0,
 'c14': 5.0990195135927845,
 'c15': 5.385164807134504,
 'c16': 5.830951894845301,
 'c2': 1.0,
 'c3': 2.0,
 'c4': 3.0,
 'c5': 1.0,
 'c6': 1.4142135623730951,
 'c7': 2.23606797749979,
 'c8': 3.1622776601683795,
 'c9': 4.0}

# TODO next

## Define Teams
TEAMS
Team1: P1,P2,P3
Team2: P4, P5
Team3: P6,P7,P8

## Define Team Assignment to initial seats
Solution1: T1->a1=P1, a2=P2,  b1=P3  (team assignment)
                 T2-> a4=P4, a3=P5
                 T3-> b3=P6,b4=P7,bb2=P8
T1 (7.3)
T2 (1)
T3 (4 )
Total unhappiness =  12.3

Solution2:
T1(4)
T3(4)
T2(10)
TU = 18

## Port functions from pseudo code to Python
unhappiness_function_per_team =  avg. distance between members
n(n-1)/2 = number of distances we need to calculate

Pseudocode:
inputs: Distance MAP, unhappy_func, Teams
def team_unhappiness(team, team_assignment,  distance_map):
    all_distances = []
    for seat, person_pivot in team_assigment:
        for other_person in team:
                  all_distances.append( MAP[person_pivot, other_person] )
    return average(all_distances)


def opt_team_unhappiness(team, team_assignment,  distance_map):
    all_distances = [ ] 
    team_2x2_combinations = combine_2_2(team)
    for p1, p2 in team_2x2_combinations:
         all_distances.append( MAP[p1,p2] )
    return average(all_distances)

 
Minimize Unhappiness function

## Create Genetic Algorithm to optimize happiness function